## Datastructuring


>At first we import the csv file made by scraping the Vivino.com social network, and `drop_duplicates`, so we only have unique wines.

In [28]:
import pandas as pd
import re

wine_data = pd.read_csv('vivino_v7.csv',index_col=[0])
print('The lengt of the rawdata: ',len(wine_data))
wine_data = wine_data.drop_duplicates(['link']).reset_index(drop=True)
print('After dropping duplicates by link:',len(wine_data))
wine_data['Year']=[i[-4:] for i in wine_data.name]
a=0
for i in wine_data['name']:
    wine_data['name'][a]=re.sub(r"\d","",i)
    a+=1

vivino_data = pd.read_csv('8385_v2.csv', index_col=[0])

print('We got 2 dataframes with (rows,columns): \n', str(wine_data.shape),'and',str(vivino_data.shape))

display(wine_data.head())
display(vivino_data.head())
print(vivino_data.isna().sum())

The lengt of the rawdata:  8575
After dropping duplicates by link: 8385


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


We got 2 dataframes with (rows,columns): 
 (8385, 5) and (8385, 11)


,name,rating,region,link,Year
0,Raro Cabernet Sauvignon Selezione,4.9,Veneto,https://www.vivino.com/marion-raro-cabernet-sa...,2007
1,Barolo Riserva Monfortino,4.9,Barolo,https://www.vivino.com/giacomo-conterno-barolo...,1985
2,La Fabriseria Amarone della Valpolicella Class...,4.9,Amarone della Valpolicella Classico,https://www.vivino.com/agricola-flli-tedeschi-...,2003
3,Barolo Lazzarito,4.9,Barolo,https://www.vivino.com/vietti-barolo-lazzarito...,2000
4,Basaltico Amarone della Valpolicella,4.9,Amarone della Valpolicella,https://www.vivino.com/ernesto-ruffo-basaltico...,2004


,price,total_ratings,total_reviews,light-bold,smooth-tannic,dry-sweet,soft-acidic,alcohol_pct,residual_sugar_g/l,acidity_g/l,winery
0,2050.86 kr.,75,11.0,72.2130,40.0278,18.00000,20.8148,NaN,NaN,NaN,Marion
1,11500 kr.,49,405.0,61.6076,80.2688,9.65777,82.9995,NaN,NaN,NaN,Giacomo Conterno
2,1100 kr.,40,83.0,85.0000,27.6326,46.49700,27.5195,NaN,NaN,NaN,Tedeschi
3,1251.36 kr.,37,382.0,61.3094,77.8291,7.14819,82.4921,NaN,NaN,NaN,Vietti
4,825 kr.,30,24.0,85.0000,19.9018,47.92760,20.3641,NaN,NaN,NaN,Ernesto Ruffo


price                   56
total_ratings            0
total_reviews          135
light-bold             133
smooth-tannic          133
dry-sweet              133
soft-acidic            133
alcohol_pct           3627
residual_sugar_g/l    7930
acidity_g/l           7990
winery                   5
dtype: int64


### **The Rawdata:**
The two DataFrames match by index. Therefore, we can merge them horizotal to one DataFrame. They have the same amount of rows, so there will be no problems merging the two DataFrame. One thing there is cathing the eye is the three columns, `alcohol_pct`, `residual_sugar_g/l` and `acidity_g/l`. There are a lot of missing values, therefore the columns are removed.  

In [33]:
from numpy import nan as NA

def data_structuring(df1,df2):
    data = pd.merge(df1 , df2 , left_index=True, right_index=True)
    
    #Delete 'kr.' from price 
    data.price = [str(i) for i in data.price]
    data.price = [i[:-3] for i in data.price]
       
    #Drop unessesary columns (alcohol_pct, residual_sugar_g/l, acidity_g/l)
    data.drop(['link','alcohol_pct', 'residual_sugar_g/l','acidity_g/l'], inplace=True, axis=1)
    
    #Change column names
    data.columns = ['name', 'rating','sub_region', 'year','price_DKK','total_ratings','total_reviews', 'light-bold','smooth-tannic','dry-sweet','soft-acidic','winery']


    #Columns(light-bold, smooth-tannic, dry-sweet, soft-acidic)-2.5
    columns = ['light-bold','smooth-tannic','dry-sweet', 'soft-acidic']
    for x in range(0,4):
        data[columns[x]]=[i-2.5 for i in data[columns[x]]]
    
    #i<0 then i=0 in the columns (light-bold, smooth-tannic, dry-sweet, soft-acidic)
    for i in range(0,len(data)):
        for j in range(0,4):
            if data[columns[j]][i]<0:
                data[columns[j]][i]=0

    #Categories
    data['light-bold-cat']=pd.cut(data['light-bold'],4,labels=["light","semi light","semi bold","bold"])
    data['smooth-tannic-cat']=pd.cut(data['smooth-tannic'],4,labels=["smooth","semi smooth","semi tannic","tannic"])
    data['dry-sweet-cat']=pd.cut(data['dry-sweet'],4,labels=["dry","semi dry","semi sweet","sweet"])
    data['soft-acidic-cat']=pd.cut(data['soft-acidic'],4,labels=["soft","semi soft","semi acidic","acidic"])
    
    #columns/100
    for j in range(0,4):
        data[columns[j]]=data[columns[j]].div(100)
    
    return data 

data = data_structuring(wine_data,vivino_data)
print('Now we got a dataframe with (rows, columns): ', str(data.shape))
display(data.head())

#region.csv 
df_regions = pd.read_csv("Regions.csv",sep=";")
df_regions.columns = ['sub_region','region','cardinal_1', 'cardinal_2']
display(df_regions.head())


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Now we got a dataframe with (rows, columns):  (8385, 16)


,name,rating,sub_region,year,price_DKK,total_ratings,total_reviews,light-bold,smooth-tannic,dry-sweet,soft-acidic,winery,light-bold-cat,smooth-tannic-cat,dry-sweet-cat,soft-acidic-cat
0,Raro Cabernet Sauvignon Selezione,4.9,Veneto,2007,2050.86,75,11.0,0.697130,0.375278,0.155000,0.183148,Marion,bold,semi smooth,semi dry,soft
1,Barolo Riserva Monfortino,4.9,Barolo,1985,11500,49,405.0,0.591076,0.777688,0.071578,0.804995,Giacomo Conterno,semi bold,tannic,dry,acidic
2,La Fabriseria Amarone della Valpolicella Class...,4.9,Amarone della Valpolicella Classico,2003,1100,40,83.0,0.825000,0.251326,0.439970,0.250195,Tedeschi,bold,smooth,sweet,semi soft
3,Barolo Lazzarito,4.9,Barolo,2000,1251.36,37,382.0,0.588094,0.753291,0.046482,0.799921,Vietti,semi bold,tannic,dry,acidic
4,Basaltico Amarone della Valpolicella,4.9,Amarone della Valpolicella,2004,825,30,24.0,0.825000,0.174018,0.454276,0.178641,Ernesto Ruffo,bold,smooth,sweet,soft


,sub_region,region,cardinal_1,cardinal_2
0,Abruzzo,Abruzzo,East,East
1,Cerasuolo d’Abruzzo,Abruzzo,East,East
2,Colline Teramane Montepulciano d’Abruzzo,Abruzzo,East,East
3,Montepulciano d'Abruzzo Colline Teramane,Abruzzo,East,East
4,Montepulciano d'Abruzzo,Abruzzo,East,East


### The Region Data:
The Region Data is read from a csv file. We see it has 4 columns, and we will use it to find the region on every italien red wine in our DataFrame. The two Dataframe will be merge using the `pd.merge(left, right, how='left')`.

### Missing Data:
In the process above we are removing all of the missing data, first by replacing the blank values with NaN with `pd.replace()`, and afterwards removing the rows woth missing data using `pd.dropna()` and `pd.notna()`. In the process we have diplayed how many missing values there are in each column, and we only have 4 missing values in the column, `winery`, and that is fine, because we don't use the winery column in the Machine Learning model.

### float and int: 
In the final dataset it is very important to have the digit indexes as `float`. and the number of ratings we convert to an `int` using `astype()`. 

In [54]:
#Merge data with regions
df_cleaned = pd.merge(data,df_regions, on='sub_region',how='left')
display(df_cleaned.head())

#drop missing data
df_na = df_cleaned.replace("", NA)
df_na = df_na.replace("N.V.", NA)
print('We have to find alle the missing values, and delete them:\n')
print(df_na.isna().sum())
df_cleaned = df_na.dropna(subset = ['region']).reset_index(drop=True)
print('\nWe drops the NA values in the region column:')
print(df_cleaned.isna().sum())
print('\nAnd afterwards in the rest of the columns with a for loop:')

columns =['price_DKK','light-bold','smooth-tannic','dry-sweet','soft-acidic','year','rating']
for j in range(0,7):
    df_cleaned = df_cleaned[df_cleaned[columns[j]].notna()]
display(df_cleaned.isna().sum())

df_cleaned['total_reviews']=df_cleaned['total_reviews'].astype(int)

for i in range(0,5): 
    df_cleaned[columns[i]]=df_cleaned[columns[i]].astype(float)



df_cleaned = df_cleaned.sort_values(by=['year'], ascending=False).reset_index(drop=True)
print('\nWe are now left with the final dataset (rows, columns):\n', str(df_cleaned.shape))
df_cleaned.head()

,name,rating,sub_region,year,price_DKK,total_ratings,total_reviews,light-bold,smooth-tannic,dry-sweet,soft-acidic,winery,light-bold-cat,smooth-tannic-cat,dry-sweet-cat,soft-acidic-cat,region,cardinal_1,cardinal_2
0,Raro Cabernet Sauvignon Selezione,4.9,Veneto,2007,2050.86,75,11.0,0.697130,0.375278,0.155000,0.183148,Marion,bold,semi smooth,semi dry,soft,Veneto,North,North-east
1,Barolo Riserva Monfortino,4.9,Barolo,1985,11500,49,405.0,0.591076,0.777688,0.071578,0.804995,Giacomo Conterno,semi bold,tannic,dry,acidic,Piemonte,North,North-west
2,La Fabriseria Amarone della Valpolicella Class...,4.9,Amarone della Valpolicella Classico,2003,1100,40,83.0,0.825000,0.251326,0.439970,0.250195,Tedeschi,bold,smooth,sweet,semi soft,Veneto,North,North-east
3,Barolo Lazzarito,4.9,Barolo,2000,1251.36,37,382.0,0.588094,0.753291,0.046482,0.799921,Vietti,semi bold,tannic,dry,acidic,Piemonte,North,North-west
4,Basaltico Amarone della Valpolicella,4.9,Amarone della Valpolicella,2004,825,30,24.0,0.825000,0.174018,0.454276,0.178641,Ernesto Ruffo,bold,smooth,sweet,soft,Veneto,North,North-east


We have to find alle the missing values, and delete them:

name                   0
rating                 0
sub_region             0
year                  17
price_DKK             56
total_ratings          0
total_reviews        135
light-bold           133
smooth-tannic        133
dry-sweet            133
soft-acidic          133
winery                 5
light-bold-cat       133
smooth-tannic-cat    133
dry-sweet-cat        133
soft-acidic-cat      133
region               324
cardinal_1           324
cardinal_2           324
dtype: int64

We drops the NA values in the region column:
name                   0
rating                 0
sub_region             0
year                  11
price_DKK             53
total_ratings          0
total_reviews        114
light-bold           114
smooth-tannic        114
dry-sweet            114
soft-acidic          114
winery                 4
light-bold-cat       114
smooth-tannic-cat    114
dry-sweet-cat        114
soft-acidic-cat      114
region 

name                 0
rating               0
sub_region           0
year                 0
price_DKK            0
total_ratings        0
total_reviews        0
light-bold           0
smooth-tannic        0
dry-sweet            0
soft-acidic          0
winery               4
light-bold-cat       0
smooth-tannic-cat    0
dry-sweet-cat        0
soft-acidic-cat      0
region               0
cardinal_1           0
cardinal_2           0
dtype: int64


We are now left with the final dataset (rows, columns):
 (7895, 19)


,name,rating,sub_region,year,price_DKK,total_ratings,total_reviews,light-bold,smooth-tannic,dry-sweet,soft-acidic,winery,light-bold-cat,smooth-tannic-cat,dry-sweet-cat,soft-acidic-cat,region,cardinal_1,cardinal_2
0,Passivento Rosso,3.6,Terre Siciliane,2019,93.17,96,1103,0.744740,0.330456,0.442528,0.210002,Barone Montalto,bold,semi smooth,sweet,soft,Sicilia,South,South
1,Pillastro Primitivo Puglia,4.0,Puglia,2019,69.00,823,2072,0.654435,0.154071,0.387081,0.066322,Cantine due Palme,semi bold,smooth,semi sweet,soft,Puglia,East,South-east
2,Fantini Sangiovese Terre di Chieti,3.6,Terre di Chieti,2019,84.17,70,1001,0.424077,0.328759,0.065716,0.403993,Farnese,semi light,semi smooth,dry,semi soft,Abruzzo,East,East
3,Appassite Negroamaro - Primitivo,4.0,Puglia,2019,89.17,102,800,0.825000,0.334615,0.349416,0.184099,Luna Argenta,bold,semi smooth,semi sweet,soft,Puglia,East,South-east
4,Passivento Rosso,3.6,Terre Siciliane,2019,93.17,97,1103,0.744727,0.330469,0.442570,0.210016,Barone Montalto,bold,semi smooth,sweet,soft,Sicilia,South,South


### Saving Dataframe
At the end we save the dataframe as a CSV file. 

In [51]:
#save dataframe as cvs file
df_cleaned.to_csv('merged_data.csv')